# Transfer Learning
This will build half of the autoencoder and pre-load with weights from the full autoencoder

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

SAVE_FOLDER='/tmp/cae_cnn_transfer'

# Delete directory if exist
if os.path.exists(SAVE_FOLDER):    
    os.system("rm -rf " + SAVE_FOLDER)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/Users/leonardo_a/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
from models import CAE_CNN_Encoder
model = CAE_CNN_Encoder()
model_in = model.input
model_out = model.output
labels_in = model.labels

### Select layers

In [3]:
# Get all convs weights
list_convs = [v for v in tf.global_variables() if "conv" in v.name]
# Get fc1 and logits
list_fc_layers = [v for v in tf.global_variables() if "fc" in v.name or "logits" in v.name]

In [4]:
tf.global_variables()

[<tf.Variable 'conv1/kernel:0' shape=(5, 5, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv2/kernel:0' shape=(5, 5, 16, 32) dtype=float32_ref>,
 <tf.Variable 'conv2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'fc1/kernel:0' shape=(1568, 200) dtype=float32_ref>,
 <tf.Variable 'fc1/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'logits/kernel:0' shape=(200, 10) dtype=float32_ref>,
 <tf.Variable 'logits/bias:0' shape=(10,) dtype=float32_ref>]

### Define loaders to load and save weights

In [5]:
# Define the saver object to load only the conv variables
saver_load_autoencoder = tf.train.Saver(var_list=list_convs)

# Define saver object to save all the variables during training
saver = tf.train.Saver()

### Define loss for Classification

In [6]:
with tf.name_scope("LOSS"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model_out, labels=labels_in))

correct_prediction = tf.equal(tf.argmax(model_out,1), tf.argmax(labels_in,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    

# Solver configuration
with tf.name_scope("Solver"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, var_list=list_fc_layers)

### Build Graph

In [7]:
# Initialize variables
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.200)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

### Load Weights

In [8]:
# Restore only the weights (From AutoEncoder)
saver_load_autoencoder.restore(sess, "/tmp/cae_cnn/model.ckpt-34")

INFO:tensorflow:Restoring parameters from /tmp/cae_cnn/model.ckpt-34


### Add some tensors to observe on Tensorboard

In [9]:
tf.summary.image("input_image", model.image_in, 4)
tf.summary.scalar("loss", loss)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(SAVE_FOLDER)
writer.add_graph(sess.graph)

### Train

In [10]:
num_epoch = 200
batch_size = 10
for epoch in range(num_epoch):
    for i in range(int(mnist.train.num_examples / batch_size)):
        # Get batch of 50 images
        batch = mnist.train.next_batch(batch_size)

        # Dump summary
        if i % 5000 == 0:            
            # Other summaries
            s = sess.run(merged_summary, feed_dict={model_in:batch[0], labels_in:batch[1]})
            writer.add_summary(s,i)                     


        # Train actually here (Also get loss value)            
        _, val_loss, t_acc = sess.run((train_step, loss, accuracy), feed_dict={model_in:batch[0], labels_in:batch[1]})                

        #if i % 5000:
            #print('Generation loss: %d, latent loss: %d' % (np.mean(gen_loss), np.mean(lat_loss)))
    print('Epoch: %d/%d loss:%d' % (epoch, num_epoch, val_loss))
    print('Save model:', epoch)
    saver.save(sess, os.path.join(SAVE_FOLDER, "model.ckpt"), epoch)

Epoch: 0/200 loss:0
Save model: 0
Epoch: 1/200 loss:1
Save model: 1
Epoch: 2/200 loss:0
Save model: 2
Epoch: 3/200 loss:0
Save model: 3
Epoch: 4/200 loss:0
Save model: 4
Epoch: 5/200 loss:0
Save model: 5
Epoch: 6/200 loss:0
Save model: 6
Epoch: 7/200 loss:0
Save model: 7


KeyboardInterrupt: 